In [1]:
# reference
# https://towardsdatascience.com/getting-census-data-in-5-easy-steps-a08eeb63995d
# Variable list
# Detailed table: https://api.census.gov/data/2018/acs/acs5/profile/variables.html
# Subject table: https://api.census.gov/data/2018/acs/acs5/subject/variables.html

In [1]:
#manipulate dataframes in python
import pandas as pd

#make API calls with python
import requests

#allows us to store results of API call cleanly
import json

In [2]:
#display all the columns in dataframe
pd.set_option('display.max_columns', None)

In [3]:
# Import the variable list that contains the ACS codes of the attributes we want to acquire using API and new attribute name
# Since the API varies by table, this file works for attributes in data profile table
df = pd.read_csv('ACS Variables_Tract_DP.csv')

In [4]:
# Select ACS codes and turn it into a list
var_codes = df['Code'].tolist()

# Select attribute name and turn it into a list, this can be self-defined. You can also choose not to rename the variable
names = df['Name in Database'].tolist()

# Zip the lists above into a dictionary to map the code with attribute name
var_name_dct = dict(zip(var_codes, names)) 

In [5]:
# Define a function to take year, code and attribute name and acquire value using Census API
def getAPIdata(year, code, AttrName):
    apiKey = '09e7e4a144645fc4e247d25e6477d1a245f561cd' # To use census API, you will need to apply for a key. Please replace the key with yours here
    year = year # Take the data at that year
    var_code = code # Take the attribute using the code
    
    # This is the base API, here we set Hennepin County with code 27 & 053, and the %s are placeholder to take time, attribute code and key
    baseAPI = 'https://api.census.gov/data/%s/acs/acs5/profile?get=%s&for=tract:*&in=state:27&in=county:053&key=%s'
    calledAPI = baseAPI % (year, var_code, apiKey) # Put year, attribute code and key into the placeholders
    response = requests.get(calledAPI) # Call API
    formattedResponse = json.loads(response.text) # Load the data into Json format
    
    data = pd.DataFrame(columns=[AttrName, 'state', 'county', 'tract'], data=formattedResponse) # Transform the data into DataFrame, and select the columns we want
    data_by_Tract  = data[reversed(data.columns)] # Reverse the order of columns
    data_by_Tract[['state', 'county', 'tract']] = data_by_Tract[['state', 'county', 'tract']].astype(str) # Transform the cell value from integar into string format
    data_by_Tract['TractId'] =data_by_Tract[['state', 'county', 'tract']].agg(''.join, axis=1) # Join state, county and tract code into full TractId
    data_by_Tract = data_by_Tract.iloc[1: ,[4,3]] # Select the data we need. We skip first row, and take column 3, 4
    
    return data_by_Tract

In [6]:
# Load the empty DataFrame that only contains TractId
All_data_by_Tract_2018 =  pd.read_csv('Tract_FIPS_codes.csv', dtype= 'str')

In [7]:
# Use a for loop to iterate through all the ACS codes we want
for key, value in var_name_dct.items():
    data_by_Tract_2018 = getAPIdata(2018, key, value)
    All_data_by_Tract_2018 = pd.merge(All_data_by_Tract_2018, data_by_Tract_2018, on='TractId') # Merge the new attribute to the existing DataFrame

In [8]:
# Import the variable list that contains the ACS codes of the attributes we want to acquire using API and new attribute name
# Since the API varies by table, this file works for attributes in subject table
# We have subject variables in "ACS Variables_Tract_Subject_SNAP.csv" and "ACS Variables_Tract_Subject.csv"
df3 = pd.read_csv('ACS Variables_Tract_Subject_SNAP.csv')
# Select ACS codes and turn it into a list
var_codes_sub = df3['Code'].tolist()
# Select attribute name and turn it into a list, this can be self-defined. You can also choose not to rename the variable
names_sub = df3['Name in Database'].tolist()
# Zip the lists above into a dictionary to map the code with attribute name
var_name_sub_dct = dict(zip(var_codes_sub, names_sub)) 

In [9]:
# Define a function to take year, code and attribute name and acquire value using Census API
# This function is used to get data from subject table
# Please refer to getAPI function above for the detailed code explanation
def getAPIdata_sub(year, code, AttrName):
    apiKey = '09e7e4a144645fc4e247d25e6477d1a245f561cd'
    year = year
    var_code = code
    baseAPI = 'https://api.census.gov/data/%s/acs/acs5/subject?get=%s&for=tract:*&in=state:27&in=county:053&key=%s'
    calledAPI = baseAPI % (year, var_code, apiKey)
    response = requests.get(calledAPI)
    formattedResponse = json.loads(response.text)
    data = pd.DataFrame(columns=[AttrName, 'state', 'county', 'tract'], data=formattedResponse)
    data_by_Tract  = data[reversed(data.columns)]
    data_by_Tract[['state', 'county', 'tract']] = data_by_Tract[['state', 'county', 'tract']].astype(str)
    data_by_Tract['TractId'] =data_by_Tract[['state', 'county', 'tract']].agg(''.join, axis=1)
    data_by_Tract = data_by_Tract.iloc[1: ,[4,3]]
    
    return data_by_Tract

In [10]:
# Use a for loop to iterate through all the ACS codes we want
for key, value in var_name_sub_dct.items():
    data_by_Tract_2018 = getAPIdata_sub(2018, key, value)
    All_data_by_Tract_2018 = pd.merge(All_data_by_Tract_2018, data_by_Tract_2018, on='TractId') # Merge the new attribute to the existing DataFrame

In [12]:
# Save that dataframe to a CSV spreadsheet
All_data_by_Tract_2018.to_csv('All_data_by_Tract_18.csv', index=False)